# 💾 System.Buffers

__What__
- ArrayPool<T> → Manages reusable arrays to reduce GC overhead
- MemoryPool<T> → Manages reusable memory blocks for high-performance scenarios
- IMemoryOwner<T> & IBufferWriter<T> → Interfaces for efficient memory handling and writing

__When__
- When performance matters: Avoids frequent allocations and garbage collection
- When working with large data chunks: Optimized for large buffers (e.g., image/video processing)
- When handling networking or serialization: Ideal for socket programming, file I/O, and streaming

In [ ]:
using System.Buffers;
using System.Net.Http;
using System.Text.RegularExpressions;

async Task<string> DownloadPageAsync(string url)
{
    using HttpClient client = new();
    return await client.GetStringAsync(url);
}

string[] ExtractWords(string content) =>
    Regex.Matches(content, @"\b[a-zA-Z]{3,}\b")  // Extract words with 3+ letters
        .Select(m => m.Value.ToLower())          // Convert to lowercase
        .Distinct()
        .ToArray();

bool IsPalindrome(string word)
{
    int length = word.Length;
    char[] buffer = ArrayPool<char>.Shared.Rent(length);

    try
    {
        word.CopyTo(0, buffer, 0, length);

        for (int i = 0; i < length / 2; i++)
        {
            if (buffer[i] != buffer[length - 1 - i])
                return false;
        }
        return true;
    }
    finally
    {
        ArrayPool<char>.Shared.Return(buffer);
    }
}

string url = "https://en.wikipedia.org/wiki/Palindrome"; // Example Wikipedia page
string content = await DownloadPageAsync(url);
string[] words = ExtractWords(content);

Console.WriteLine("Palindromic words found:");
foreach (var word in words)
{
    if (IsPalindrome(word))
        Console.WriteLine(word);
}

__Why__
- ✅ Reduces GC Pressure → Reusing the buffer avoids excessive allocations
- ✅ Improves Performance → Efficient memory reuse speeds up file processing
- ✅ Scalable for Large Files → Handles gigabytes of data without excessive memory overhead

In [ ]:
using System.Buffers;

using(MemoryPool<byte> pool = MemoryPool<byte>.Shared)
using(IMemoryOwner<byte> memoryOwner = pool.Rent(1024)) // Rent a 1 KB buffer
{
    Memory<byte> memory = memoryOwner.Memory;
    Console.WriteLine($"Allocated buffer size: {memory.Length}");

    Span<byte> span = memory.Span;
    span[0] = 42; // Works fine

    Console.WriteLine($"First byte: {span[0]}");
} // Buffer is automatically returned to the pool when memoryOwner is disposed

In [ ]:
// Rust equivalent

let mut buffer = vec![0u8; 1024]; // Heap-allocated buffer
{
    let slice: &mut [u8] = &mut buffer[..]; 
    slice[0] = 42; // Works fine
    println!("First byte: {}", slice[0]);
} // Borrowing ends here, no memory leaks.

In [ ]:
using System.Buffers;

class Testing
{
    public static void Test()
    {
        byte[] data = Encoding.UTF8.GetBytes("123456789");
        ReadOnlyMemory<byte> memory = new(data);

        ReadOnlySequence<byte> sequence = new(memory);
        SequenceReader<byte> reader = new(sequence);

        if (reader.TryRead(out byte value))
            Console.WriteLine($"First byte: {value}");
    }
}

Testing.Test()

- __`MemoryPool<T>`__ Efficient memory management for large buffers
    - When working with high-throughput systems (e.g., custom network protocols)
    - Ideal for large memory blocks or high-performance pipelines (e.g., networking, serialization)
- __`IBufferWriter<T>`__ High-performance buffered writing
    - Useful for serialization, network responses, and streaming large data without excessive allocations
    - When you need efficient writing (e.g., network buffers, logging, serialization)
    - When performance is critical in data transformation pipelines
- __`ReadOnlySequence<T>`__ Efficient reading over large, segmented data
    - Helps process large data chunks efficiently without copying
    - Used in high-performance networking (e.g., Kestrel, System.IO.Pipelines)
    - Processing network packets without creating extra allocations
    - High-performance log parsing or binary protocol handling
- __`SequenceReader<T>`__ Stream-like processing of `ReadOnlySequence<T>`

- https://learn.microsoft.com/en-us/dotnet/standard/io/buffers

<img src=images/ro-sequence.png width=1000>

# 🧠 Generative AI

<img src=images/model-function-calling.jpg width=800>

- https://github.com/xoredge/ai-tools-demo
- Ollama
    - https://ollama.com
- Semantic Kernel
    - https://learn.microsoft.com/en-us/semantic-kernel/overview
    - https://github.com/microsoft/semantic-kernel
- Function Calling
    - https://platform.openai.com/docs/guides/function-calling
    - https://ollama.com/blog/tool-support

## 🎁 Examples

- Multi Agent
    - https://github.com/khurram-uworx/dotnetdebrief/blob/main/src/ChatBots/AgentTicketHandler.cs
    - YAML
    - We can define using Code
        - We can have Parameters
    - Chain of Thoughts
    - Function Calling and why we need "Vectorization" (Plugin)
- Notepad
    - https://github.com/khurram-uworx/dotnetdebrief/tree/main/src/Notepad
    - Try prompt engineering to simulate Notepad AI use cases
    - Show the System Prompt

## 📖 Grounding

In [ ]:
you are a sql expert who creates sql server queries to answer the user inputs

use the following sql server database schema information to create queries
- database has Users table that has id (int), username (string), password (string) and isactive(bool) fields
- database has Transactions table that has id(int) foreign key to Users id field, date (datetime), amount (int) and remark (string) fields

Important; just write sql query in answer; dont explain anything
Very Important; dont forget the underlying database is sql server; use appropriate tsql syntax

User input: Who are my active users that have made some transactions in the last month?

In [ ]:
User input: How many active users do we have?
User input: Who are my active users that have made some transactions in the last month?
User input: Who are my top 5 active users by transaction amount?

In [ ]:
The movie Inception (2010), directed by Christopher Nolan, is about a skilled thief named Dom Cobb who specializes in extracting secrets
from people's subconscious during dreams. The film explores the concept of dreams within dreams and questions the nature of reality.

Given this information, write a short review of Inception that highlights its themes and storytelling

In [ ]:
The Matrix (1999) is about a computer hacker, Neo, who discovers that reality is a simulated world controlled by machines.
The Truman Show (1998) follows Truman Burbank, who slowly realizes his entire life has been a reality TV show. Both films explore the theme of questioning reality.

Using this information, compare how The Matrix and The Truman Show depict the idea of escaping a false reality

In [ ]:
You are an AI that recommends family-friendly movies for a weekend watch. Here are five movies retrieved from a database based on relevance to the request

Finding Nemo (2003) - A heartwarming animated film about a father fish searching for his lost son.
Toy Story 3 (2010) - A story about friendship, change, and growing up, featuring beloved toy characters.
Paddington 2 (2017) - A charming and humorous film about a bear in London with a strong message of kindness.
The Incredibles (2004) - A superhero movie with action, humor, and family dynamics.
How to Train Your Dragon (2010) - A touching story about friendship between a boy and a dragon.

Based on this retrieved list, craft a response recommending the best option for a family movie night.

## 💡 Retreival Augmented Generation (RAG)

<img src=images/rag-sequence-diagram.png width=700>

__Resumes RAG__

<img src=images/resumes-rag.png>

- https://github.com/khurram-uworx/dotnetdebrief/blob/main/src/ChatBots/KernelMemoryPgRagSK.cs

In [ ]:
Using the education details and graduation year from the provided resumes, estimate each candidate’s years of experience.
If a candidate graduated in 2020, assume they have been working since then. Given that the current year is 2025,
this would mean they have approximately 5 years of experience.

Identify and list candidates who have 5 or more years of experience based on this approach. If other work experience
details are available, incorporate them to refine the estimation. Provide a ranked list of qualified candidates along
with their estimated years of experience.

__Concepts__

- Structured Outputs
    - https://platform.openai.com/docs/guides/structured-outputs
    - https://ollama.com/blog/structured-outputs
    - https://devblogs.microsoft.com/semantic-kernel/using-json-schema-for-structured-output-in-net-for-openai-models
- Embeddings
    - https://openai.com/index/introducing-text-and-code-embeddings
    - https://openai.com/index/new-embedding-models-and-api-updates

- Importance of Vector Search / Semantic Search


<img src=images/agentic-rag.png width=700>

- https://vectorize.io/how-i-finally-got-agentic-rag-to-work-right

__Resources__
- https://help.openai.com/en/articles/8868588-retrieval-augmented-generation-rag-and-semantic-search-for-gpts
- https://cloud.google.com/use-cases/retrieval-augmented-generation
- https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation
- https://en.wikipedia.org/wiki/Retrieval-augmented_generation

## 🚀 Gen AI Applications

<img src=images/genai-apps.png>

- AlphaFold Protein Structure Database https://alphafold.ebi.ac.uk
    - https://www.youtube.com/watch?v=P_fHJIYENdI The Most Useful Thing AI Has Ever Done

## 📚 Resources

- https://devblogs.microsoft.com/dotnet/announcing-generative-ai-for-beginners-dotnet
    - https://github.com/microsoft/Generative-AI-for-beginners-dotnet
    - https://www.youtube.com/playlist?list=PLdo4fOcmZ0oUkPkWy9EtmBvFAjfp4ZHnx

<img src=images/gen-ai-youtube.png>

# ⏭️ Where To Next?

## 🧠 RAG: Where we are

<img src=images/sell-me-this-pen.jpg width=400>
<img src=images/ai-startup.webp width=400>

<img src=images/chat-template.png>

- https://devblogs.microsoft.com/dotnet/announcing-dotnet-ai-template-preview1

## 💡 Where to next

<img src=images/rag-overview.png width=1000>

<img src=images/lego-pieces-basic.JPG>

- Foundation Models
- Grounding
- Function Calling
- RAG; Vector Database, Embedding Models

<img src=images/sk-arch-1.png width=800><br>
<img src=images/sk-arch-2.png width=800>

- https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent
- https://learn.microsoft.com/en-us/semantic-kernel/frameworks/process/process-framework ⚠️

- https://learn.microsoft.com/en-us/dotnet/ai/tutorials/llm-eval
- https://learn.microsoft.com/en-us/shows/mr-maedas-cozy-ai-kitchen 👈
    - https://www.youtube.com/playlist?list=PLlrxD0HtieHjHoXHYSiSvpTp_sE5JhNEE

- https://azure.github.io/ai-app-templates 👈

<img src=images/langchain-arch.webp width=800>

## 🎈 Software Engineering Spin

### 🚀 Semantic Kernel Plugins

- Semantic Kernel Search Plugin using Vector Database
    - https://learn.microsoft.com/en-us/semantic-kernel/concepts/text-search/text-search-plugins
    - https://learn.microsoft.com/en-us/semantic-kernel/concepts/text-search/text-search-vector-stores

<img src=images/sk-search.png>

### 🚀 Model Context Protocol

- https://www.anthropic.com/news/model-context-protocol
- https://github.com/modelcontextprotocol
    - https://github.com/modelcontextprotocol/csharp-sdk
    - https://github.com/modelcontextprotocol/servers

__Semantic Kernel__
- MCP Tools as Kernel Functions
- https://github.com/microsoft/semantic-kernel/blob/main/dotnet/samples/Demos/ModelContextProtocolPlugin/Program.cs
    - https://devblogs.microsoft.com/semantic-kernel/integrating-model-context-protocol-tools-with-semantic-kernel-a-step-by-step-guide
    - https://devblogs.microsoft.com/semantic-kernel/building-a-model-context-protocol-server-with-semantic-kernel

<img src=images/mcp.png>

https://github.com/khurram-uworx/dotnetdebrief/blob/main/src/ChatBots/McpRepository.cs

### 🚀 Multi Agents

We explored "AutoGen" which is a Microsoft Research project for Agentic AI scenarios
    - They have recently revamped it and v0.4 is out but the new version is Python for now and .NET version will arrive soon
- Some elements of it are now in Semantic Kernel already; there are some nice higher order concepts; we can define Agents in YAML files, how multiple agents can participate in a single chat etc; unfortunately its still "Preview" and works best with OpenAI; i tried it with Ollama / Lllama 3.2 and was able to figure out few initial issues.
 
- https://devblogs.microsoft.com/semantic-kernel/empowering-ai-agents-with-tools-via-openapi-a-hands-on-guide-with-microsoft-semantic-kernel-agents
- https://devblogs.microsoft.com/semantic-kernel/guest-blog-building-multi-agent-systems-with-multi-models-in-semantic-kernel-part-1/
- https://devblogs.microsoft.com/all-things-azure/agentic-philosophers
    - https://github.com/microsoft/all-things-azure/tree/main/agentic-philosophers 👈
    - https://github.com/khurram-uworx/dotnetdebrief/blob/main/src/ChatBots/AgentDebate.cs
    - Multi-agent chat
    - Chat with documents
    - LLM Functions/Inference

<img src=images/agentic-philosphers.png>

- https://www.developerscantina.com/p/semantic-kernel-multiagents/
    - https://github.com/qmatteoq/SemanticKernel-Demos/tree/main/SemanticKernel.Agents

- https://devblogs.microsoft.com/semantic-kernel/guest-blog-creative-writing-assistant-a-multi-agent-app-sample-with-semantic-kernel-net-aspire
    - https://github.com/Azure-Samples/contoso-creative-writer
    - https://github.com/Azure-Samples/aspire-semantic-kernel-creative-writer
- https://cookbook.openai.com/examples/orchestrating_agents
    - https://github.com/openai/swarm

__Additional Resources__
- https://learn.microsoft.com/en-us/azure/ai-services/agents/overview
- https://devblogs.microsoft.com/all-things-azure/how-to-develop-ai-apps-and-agents-in-azure-a-visual-guide

### 🚀 Process Framework

- https://learn.microsoft.com/en-us/semantic-kernel/frameworks/process/process-framework
    - https://github.com/microsoft/semantic-kernel/tree/main/dotnet/samples/GettingStartedWithProcesses
    - https://devblogs.microsoft.com/semantic-kernel/integrating-ai-into-business-processes-with-the-process-framework

## 🎈 RAG - CAG

<img src=images/rag-cag.webp>

### 🚀 GraphRAG

- Scattered information
    - Relevant Chunks all over the information
- Noise
- Context Loss
- Scalability

- https://www.microsoft.com/en-us/research/blog/graphrag-unlocking-llm-discovery-on-narrative-private-data
- https://microsoft.github.io/graphrag
    - https://github.com/microsoft/graphrag

__C#__
- https://jasonhaley.com/2024/08/06/study-notes-graph-rag1-code-sample-notebook
    - https://github.com/JasonHaley/semantic-kernel-getting-started/blob/main/notebooks/1_PropertyGraphRAG.ipynb
    - https://github.com/JasonHaley/semantic-kernel-getting-started

<img src=images/graph-rag-code-translation.png width=1000>

### 🚀 Struct RAG

- https://arxiv.org/pdf/2410.08815

<img src=images/ragstruct-overview.png width=1000>

- https://github.com/kbeaugrand/KernelMemory.StructRAG

## 🚀 Beyond API Calls & Chatbots

<img src=images/useful-ai-apps.png width=1000>

https://www.youtube.com/watch?v=awztkr8n0AA
- Keynote: The past, present, and future of AI for application developers - Steve Sanderson
- __NDC Conferences__ 2025 March

<img src=images/useful-ai-webapps.png width=1000>

https://www.youtube.com/watch?v=TSNAvFJoP4M
- How to add genuinely useful AI to your webapp (not just chatbots) - Steve Sanderson
- __NDC Conferences__ 2024 July

<img src=images/this-is-not-enough.jpg>

# 🔢 ML

<img src=images/ai-ml-dl.png width=500>

Machine Learning (ML) is __a subset of artificial intelligence__ (AI) that focuses on the development of __algorithms__ that allow systems to __learn from data__, identify patterns, and make decisions with __minimal human intervention__
- It involves creating __mathematical models__ that can improve their performance as they are exposed to more data over time, effectively adapting and __improving without being explicitly programmed__ for each task

- Temperature Monitoring / Loadshedding
- User's Behavior
    - Fair Use
    - Planning to Leave
- Predictive maintenance system

__Software Engineer's View__
- Data Collection and Preprocessing: The system needs __clean, structured, and high-quality__ data. This involves gathering relevant data from sensors, databases, or other sources, followed by preprocessing steps like normalization, missing value imputation, and feature engineering
- Model Selection and Training: Different types of __machine learning models__, such as regression, classification, or clustering algorithms, need to be selected based on the nature of the problem. The model is __trained__ using historical data, and its performance is __evaluated__ using metrics such as accuracy, precision, recall, or F1 score
- Model Deployment: Once trained, the model must be __integrated into the software ecosystem for real-time use__. This could involve deploying the model to the cloud, edge devices, or even embedded systems, depending on the requirements
- Monitoring and Maintenance: ML models can degrade over time as the data distribution changes, leading to a phenomenon called __model drift__. Regular monitoring and retraining of the model with new data are essential to __maintain the system’s effectiveness__
- Scalability and Optimization: Machine learning systems often need to handle large volumes of data in real time, which requires __careful engineering to ensure scalability__. Optimizations such as model compression, parallelization, or distributed computing may be necessary

<img src=images/ml-ai-roles.png>

- https://mbmlbook.com "Model-Based Machine Learning" is a comprehensive online book that introduces readers to building machine learning models for real-world problems; Pragmatic Programmer style

- Domain Expert
- Security; data privacy
- Ethical considerations
- Bias in models, transparency of decision-making

# 📚 Resources

<img src=images/ml-models-everywhere.png>

- https://huggingface.co/models
- https://onnx.ai/models
- https://ollama.com/library
- https://ai.azure.com/explore/models
- https://aws.amazon.com/marketplace/solutions/machine-learning/pre-trained-models
- https://hub.docker.com/catalogs/gen-ai